# Using Medical Image Segmentations Dataset

## Install Dependencies

In [1]:
!pip install -q -e  ..

## Imports

In [21]:
import evalseg
import numpy as np
import pandas as pd
from ipywidgets import interact,interact_manual, IntSlider
import os

# Variables

In [3]:
root_data='../datasets'

## Select Dataset

In [23]:
@interact
def select_dataset(dataset_name=sorted([k for k in os.listdir(root_data) if os.path.isdir(f'{root_data}/{k}')])):
    dataset=evalseg.io.NibDataset(f'{root_data}/{dataset_name}')
    @interact_manual
    def select_case(case=sorted(dataset.get_available_ids())):
        gt,gt_voxel_size=dataset.get_groundtruth(case)
        ct,ct_voxel_size=dataset.get_CT(case)
        if ct is None: ct,ct_voxel_size=gt,gt_voxel_size 
        assert all(ct_voxel_size==gt_voxel_size)
        
        gtmax=(gt.sum(axis=2).sum(axis=1).argmax(),gt.sum(axis=2).sum(axis=0).argmax(),gt.sum(axis=1).sum(axis=0).argmax())              
        evalseg.ui.ortho_slicer(ct, {'gt':gt}, gtmax, spacing=gt_voxel_size)
        
        @interact
        def select_method(method=dataset.get_prediction_methods()):
            pred,pred_voxel_size=dataset.get_prediction(method,case)
            assert all(pred_voxel_size==gt_voxel_size)
            evalseg.ui.ortho_slicer(ct, {method:pred}, gtmax, spacing=gt_voxel_size)



interactive(children=(Dropdown(description='dataset_name', options=('Example', 'Task001_LiverTumor', 'Task002_…

# View one slice of all predictions

In [25]:
evalseg.reload()
@interact
def select_dataset(dataset_name=sorted(os.listdir(root_data))):
    global dataset
    dataset=evalseg.io.NibDataset(f'{root_data}/{dataset_name}')
    @interact_manual
    def select_case(case=sorted(dataset.get_available_ids())):
        dataset.load_all_of_case(case)
        gto,gto_voxelsize=dataset.get_groundtruth(case)
        cto,cto_voxelsize=dataset.get_CT(case)
        gtmax=(gto.sum(axis=2).sum(axis=1).argmax(),gto.sum(axis=2).sum(axis=0).argmax(),gto.sum(axis=1).sum(axis=0).argmax())
        @interact
        def axselect(ax=['z','x','y']):       
            axi={'x':0,'y':1,'z':2}[ax]
            @interact
            def onlyMaxFrame(frame=IntSlider(gtmax[axi],min=0,max=gto.shape[axi]-1)):
                gt, voxelsize = evalseg.geometry.slice(gto, gto_voxelsize, axi, [frame])
                ct,_=evalseg.geometry.slice(cto, None, axi, [frame])
                
                data = {p: evalseg.geometry.slice(dataset.get_prediction(p,case)[0], None, axi, [frame])[0] for p in dataset.get_prediction_methods()}
                data={'GroundTruth':gt,**data}
                evalseg.ui.multi_plot_2d(
                    ct,
                    data,
                    spacing=voxelsize,
                    args={
                        "imglabel": f"{dataset_name.split('_')[1]} {case} {ax}={frame}",
                        'z_titles':[frame],
                        "add_backimg": 1,
                        "clahe": 1,
                        "crop2roi": 1,
                        "zoom2segments": 1,
                        "add_notzoom_img": 1,
                        #                 'interactive':1,
                        "show_tp_fp_fn": 0,
                        "show": 1,
                        # "dst": f"{outdir2}/allmethods",
                        "spacing": voxelsize,
                    },
                )


interactive(children=(Dropdown(description='dataset_name', options=('Example', 'README.MD', 'Task001_LiverTumo…

In [11]:
dataset.get_groundtruth('108.nii')[0]

Format,coo
Data Type,uint8
Shape,"(512, 512, 856)"
nnz,1455033
Density,0.00648424112908194
Read-only,True
Size,34.7M
Storage ratio,0.2


In [16]:
free -h

NameError: name 'free' is not defined

In [14]:
np.max(dataset.get_groundtruth('108.nii')[0])

1